In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [ ]:
!pip install konlpy
!pip install git+https://github.com/kakaobrain/pororo.git
!pip install pykrx 
!pip install jamo

In [ ]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
cd Mecab-ko-for-Google-Colab

In [ ]:
!bash install_mecab-ko_on_colab_light_220111.sh

In [7]:
from pororo import Pororo

In [ ]:
cd ../

In [ ]:
# 코스피 코스닥 상장 기업 정보 리스트 생성
from pykrx import stock
import pandas as pd

KOSDAQ = stock.get_market_ticker_list(market="KOSDAQ")
KOSPI = stock.get_market_ticker_list(market="KOSPI")

def return_name():
    Company = []

    for ticker in KOSDAQ:
        Value =stock.get_market_ticker_name(ticker)
        Company.append([Value,ticker,"KOSDAQ"]) # 기업명, 종목번호, 코스닥 리스트에 추가
    for ticker in KOSPI:
        Value =stock.get_market_ticker_name(ticker)
        Company.append([Value,ticker,"KOSPI"]) # 기업명, 종목번호, 코스닥 리스트에 추가
 
    return Company

In [10]:
# 종성 여부 체크
from jamo import h2j, j2hcj

def get_jongsung_TF(text):
  sample_text_list = list(text)
  last_word = sample_text_list[-1]
  last_word_jamo_list = list(j2hcj(h2j(last_word)))
  last_jamo = last_word_jamo_list[-1]

  jongsung_TF = "T"

  if last_jamo in ['ㅏ','ㅑ','ㅓ','ㅕ','ㅗ','ㅛ','ㅜ','ㅠ','ㅡ','ㅣ','ㅘ','ㅚ','ㅙ','ㅝ','ㅞ','ㅢ','ㅐ','ㅔ','ㅟ','ㅖ','ㅒ'] :
    jongsung_TF = "F"

  return jongsung_TF

In [11]:
# 사용자 사전 적용 : 고유명사 구분하기 위해 코스피, 코스닥 상장 기업들 사용자 사전에 추가
with open("./mecab-ko-dic-2.1.1-20180720/user-dic/nnp.csv",'r',encoding='utf-8') as f:
  file_data = f.readlines()
Company = return_name()
for c in Company:
  jongsung_TF = get_jongsung_TF(c[0])
  line = '{},,,,NNP,*,{},{},*,*,*,*,*\n'.format(c[0], jongsung_TF, c[0])
  file_data.append(line)
with open("./mecab-ko-dic-2.1.1-20180720/user-dic/nnp.csv",'w',encoding='utf-8') as f:
  for line in file_data:
    f.write(line)

In [29]:
# 뉴스 요약 및 기업 정보 추출
from bs4 import BeautifulSoup
import requests
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from konlpy.tag import Mecab

bullet_summ = Pororo(task="text_summarization", lang="ko", model="bullet")
abs_summ = Pororo(task="text_summarization", lang="ko", model="abstractive")

def summerize_news(url, Company):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    body = soup.find_all(id='dic_area')
    if len(body) > 0:
        text = body[0].text
        output_bullet = bullet_summ(text,
                                    beam=5,
                                    len_penalty=0.6,
                                    no_repeat_ngram_size=3,
                                    top_k=50,
                                    top_p=0.7)
        summary_output = abs_summ(text,
                                  beam=5,
                                  len_penalty=0.6,
                                  no_repeat_ngram_size=3,
                                  top_k=50,
                                  top_p=0.7
                                  )
      
        mecab = Mecab()
        noun = mecab.nouns(summary_output) # 명사만 추출
        pos = mecab.pos(summary_output) # 형태소 분석
        removelist = ['JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 'JX', 'JC', 'EF'] # 조사 품사 리스트 - 제거할 품사
        resultlist = []
        for p in pos :
          if len(p[0])>1 and p[1] not in removelist : # 토큰의 길이가 1 초과이고 조사가 아니면 감정 분석에 활용할 토큰이므로 리스트에 추가
            resultlist.append(p)
        #print(pos)
        #print(resultlist)

        company_name = []
        for c in Company :
          company_name.append(c[0]);

        summ = "[주요] \n"
        for line in output_bullet:
          summ += "-"+line+"\n"
        summ += "[요약] \n"
        summ += summary_output+"\n" # 뉴스 요약본
        summ += "[회사정보] \n"
        chk = False
        for n in noun : # 뉴스 요약본 속 명사들 중에서 코스피, 코스닥 상장 기업명 찾기
          try :
            idx = company_name.index(n)
          except :
            chk = False
          else :
            summ += str(Company[idx])+"\n"
            chk = True
            break
        if chk==False :
          summ += "null \n"
        print(summ)
        return summ
    else:
        print("error")
        return "error"

In [ ]:
import time
import re
import requests
from datetime import datetime
from dateutil.relativedelta import relativedelta

# 오늘 날짜 형식화
now=datetime.now()
now=str(now)
year=now[0:4]
month=now[5:7]
day=now[8:10]
res=year+month+day

# url file에 저장 (max_page_num 바꿔주면 됨)
output_file_name=str(res)+"_url.txt"
output_file = open(output_file_name, "w", encoding="utf-8")
page_num = 1
max_page_num = 1

user_agent = "'Mozilla/5.0"
headers ={"User-Agent" : user_agent}

urls = []
while page_num<=max_page_num:
    page_url = "http://news.naver.com/main/list.nhn?sid2=258&sid1=101&mid=shm&mode=LS2D&date=" + str(res) + "&page=" + str(page_num) + ""
    response = requests.get(page_url, headers=headers)
    html = response.text
    # url 추출
    url_frags = re.findall('<a href="(.*?)"',html)
    for url_frag in url_frags:
        if "article" in url_frag:
            urls.append(url_frag)
            #print(url_frag)
    page_num+=1
urls=set(urls)
for url in urls:
    print(url, file=output_file)
time.sleep(2)
output_file.close()

# url 바탕으로 요약문 생성
# 요약문 file에 저장(테스트용으로 count로 6개만 끊어준것임)
output_file_name2=str(res)+"_summarized.txt"
output_file2 = open(output_file_name2, "w", encoding="utf-8")
count=0
for url in urls:
  if(count>20):
    break
  count=count+1
  print(summerize_news(url,return_name()), file=output_file2)
output_file2.close()
'''
# mount it
from google.colab import drive
drive.mount('/content/drive')
# copy it there
!cp "20220408_summarized.txt" /content/drive/MyDrive
!cp "20220408_url.txt" /content/drive/MyDrive
'''


In [30]:
# 테스트 뉴스
#summerize_news('https://n.news.naver.com/mnews/article/023/0003688236?sid=101', return_name()) 

[('삼성전자', 'NNP'), ('의', 'JKG'), ('삼성전자', 'NNP'), ('가', 'JKS'), ('스마트폰', 'NNP'), ('반도체', 'NNG'), ('업', 'NNG'), ('황', 'NNG'), ('의', 'JKG'), ('바닥', 'NNG'), ('탈출', 'NNG'), ('시기', 'NNG'), ('에', 'JKB'), ('대한', 'VV+ETM'), ('전망', 'NNG'), ('이', 'JKS'), ('점점', 'MAG'), ('늦춰', 'VV+EC'), ('지', 'VX'), ('면서', 'EC'), ('경쟁력', 'NNG'), ('이', 'JKS'), ('정체', 'NNG'), ('돼', 'XSV+EC'), ('있', 'VX'), ('다는', 'ETM'), ('지적', 'NNG'), ('을', 'JKO'), ('받', 'VV'), ('고', 'EC'), ('있', 'VX'), ('다', 'EF'), ('.', 'SF')]
[('삼성전자', 'NNP'), ('삼성전자', 'NNP'), ('스마트폰', 'NNP'), ('반도체', 'NNG'), ('바닥', 'NNG'), ('탈출', 'NNG'), ('시기', 'NNG'), ('대한', 'VV+ETM'), ('전망', 'NNG'), ('점점', 'MAG'), ('늦춰', 'VV+EC'), ('면서', 'EC'), ('경쟁력', 'NNG'), ('정체', 'NNG'), ('다는', 'ETM'), ('지적', 'NNG')]
[주요] 
-삼성전자 시총비율 18.4%까지 추락
- 외국인 매도세 더해져
[요약] 
삼성전자의 삼성전자가 스마트폰 반도체 업황의 바닥 탈출 시기에 대한 전망이 점점 늦춰지면서 경쟁력이 정체돼 있다는 지적을 받고 있다.
[회사정보] 
['삼성전자', '005930', 'KOSPI']



"[주요] \n-삼성전자 시총비율 18.4%까지 추락\n- 외국인 매도세 더해져\n[요약] \n삼성전자의 삼성전자가 스마트폰 반도체 업황의 바닥 탈출 시기에 대한 전망이 점점 늦춰지면서 경쟁력이 정체돼 있다는 지적을 받고 있다.\n[회사정보] \n['삼성전자', '005930', 'KOSPI']\n"